## Spatial fidelity template
Infers a spatial fiedlity map for every individual from a ```.mymridon``` experiment file and saves it as a csv, which can be further analyzed in the optinal part or exported into other software such as RStudio.  
This notebook uses the following:
* the py-myrmidon library ([Documentation](https://formicidae-tracker.github.io/myrmidon/latest/))
* scipy spatial library ([Documentation](https://docs.scipy.org/doc/scipy/reference/spatial.html))

In [1]:
import py_fort_myrmidon as fm
import numpy as np  # Fundamental math library in python.
from datetime import datetime, timedelta  # For convenient handling of time and date
import pandas as pd  # To create a pandas dataframe, an equivalent to an R dataframe
from scipy.spatial import KDTree
from scipy.spatial.distance import cosine
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt  # Optional: for plorring
# Optional: makes plots interactive:
%matplotlib widget
%load_ext blackcellmagic

In [2]:
myrmidonFilepath = "dataset/example.myrmidon"
experiment = fm.Experiment.Open(myrmidonFilepath)
di = fm.Query.GetDataInformations(experiment)
t_start, t_end = di.Start, di.End 

Load a period of tracking data, here 12 hours, and get the x and y limits of the tracking cooridnates. Infer mean ant body length and use it as internal radius for the hexagons ```r_int``` to tesselate the space.

In [3]:
identifiedFrames = fm.Query.IdentifyFrames(experiment,
                                           start=t_start,
                                           end=t_start.Add(1)) # will get the first tracked frame only as we add a single nanosecond
assert len(identifiedFrames) == 1
xlim = [0, identifiedFrames[0].Width]
ylim = [0, identifiedFrames[0].Height]
ant_radius = []
for antID in experiment.Ants:
    measurements = fm.Query.ComputeMeasurementFor(experiment=experiment, antID=antID, measurementTypeID=1)
    if measurements: #discards ants without any measurements
        ant_radius.append(np.mean([m.LengthPixel for m in measurements]))
r_int = np.mean(ant_radius)  # One ant radius, arbitrary choice
xlim

Identifiying frames:   0%|                       | 0/1 [00:00<?, ?tracked min/s]


[0, 7920]

In [4]:
ylim

[0, 6004]

In [5]:
r_int

281.5809039111243

A spatial tesselation of hexagons is used to define the sites where the observations of the individuals are counted. A nearest neighbour tree (KDTree) is used with the centers of the hexgons mentioned. For a given set of coordinates, a KDTree can be used to efficiently find the seed that is closest to the input coordinates, here the waypoints of the trajectory. The hexagonal boundaries are not defined explicitly. Simply by placing the hexagonal centers accordingly (see ```method ``` variable), the resulting pattern will be hexagonal or squared. The code in the following cell computes the hexagon centers that seed the KDTree.  
This part should probably only be edited w.r.t. the method and the internal radius.

In [6]:
method = 'vertical'
#method = 'horizontal'
#method = 'square'

if method == 'vertical':
    dx = 2*r_int
    dy = np.sqrt(3)*r_int
elif method == 'horizontal':
    dx = np.sqrt(3)*r_int
    dy = 2*r_int
elif method == 'square':
    dx = 2*r_int
    dy = 2*r_int
n_x = np.diff(xlim) // dx + 1
n_y = np.diff(ylim) // dy + 1
osx = n_x * dx - np.diff(xlim)
osy = n_y * dy - np.diff(ylim)
x = np.arange(start=xlim[0]-osx/2, step=dx, stop=n_x*dx+osx/2)
y = np.arange(start=ylim[0]-osy/2, step=dy, stop=n_y*dy+osy/2)
xm, ym = np.meshgrid(x, y)
if method == 'vertical':
    xm[::2] = xm[::2] + r_int
elif method == 'horizontal':
    ym[:, 1::2] = ym[:, 1::2] + r_int
sites = np.stack([xm.flatten(), ym.flatten()], axis=1)
kdtree = KDTree(sites)

The following is an iterator for fort mymrmidon time over days in a period. See Ant metadata template for explanation.

In [7]:
def fm_time_range(
    start: fm.Time,
    end: fm.Time,
    *,
    increment=24 * fm.Duration.Hour,
):
    """Slice a time range is sub-time range 'à la' range()

    Args:
        start (fm.Time): the start time to consider
        end (fm.Time): the end time to consider
        increment (fm.Duration): the time increment to slice the whole time range

    Yields:
        Tuple[fm.Time,fm.Time]: start and end time of the sub range that span at most 'increment'
    """
    while start < end:
        last = start
        start = min(start.Add(increment), end)
        yield last, start

Count the observations of every individual in every site. In the end, the counts are normalized per individual counts where there is at least ```counts_cutoff``` times in total and set to zero for individuals that have been observed less often.

In [8]:
counts_cutoff = 0
# Create a dictionary with an array full of zeros and shape (number individuals x number sites) for each Space
counts = {}
for s in experiment.Spaces:
    counts[s] = np.zeros((len(experiment.Ants), len(sites)))
idxmap = dict(zip(experiment.Ants, range(len(experiment.Ants))))  # Maps the ant id to the matrix index

for t_begin,t_last in fm_time_range(t_start, t_end):
    trajectories = fm.Query.ComputeAntTrajectories(experiment,
                                                   start=t_begin,
                                                   end=t_last)
    for t in trajectories:
        dist, zone_indices = kdtree.query(t.Positions[:, 1:3])
        ind, cts = np.unique(zone_indices, return_counts=True)
        counts[t.Space][idxmap[t.Ant], ind] += cts
        
for s in experiment.Spaces:
    row_sums = counts[s].sum(axis=1)[:, np.newaxis]
    counts[s] = np.divide(counts[s], row_sums, where=row_sums > counts_cutoff)  # Normalize per individual
    counts[s][np.where(row_sums <= counts_cutoff),:] = 0  # Reset individuals with insufficents total counts

Computing ant trajectories:  99%|███▉| 120/121 [00:01<00:00, 70.57tracked min/s]


Create a pandas dataframe and save it to a csv which can be imported in R for example. The index will be the ant id and the column is just a site numeration. Dataframe shape: number of ants x number of sites.

In [9]:
for s in experiment.Spaces:
    df = pd.DataFrame(data=counts[s], index=experiment.Ants)
    f_count = "normalized_counts_space_{}_{}_space_{}_{}.csv".format(experiment.Name, experiment.Spaces[s].Name, t_start, t_end)
    df.to_csv(f_count)

### Optional part 1: heatmap visualization
```ind_x``` and ```ind_y``` are used to arrange the subplots. Note the black plots for individuals that where not counted often enough.

In [ ]:
nb_plot_cols = 10
ind_y = nb_plot_cols
ind_x = len(experiment.Ants) // nb_plot_cols

fig, ax = plt.subplots(ind_x,ind_y, figsize=(8,16))
for i in range(ind_x):
    for j in range(ind_y):
        ant_index = j + nb_plot_cols*i
        ax[i,j].imshow(counts[1][ant_index].reshape(xm.shape))
        ax[i,j].axis('equal')
        ax[i,j].axis('off')
        ax[i,j].set_title(ant_index+1)
fig.tight_layout()
plt.show()

## Optional part 2: ant - ant spatial fidelity similarity
Use cosine distance (other options are: correlation, euclidean, etc. see [here](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html) for ideas and documentation) to compare site count vectors among all individuals and save as a csv.

In [ ]:
distance_correlation = np.zeros((len(experiment.Ants),len(experiment.Ants)))
for i in range(len(experiment.Ants)):
    for j in range(i, len(experiment.Ants)):
        distance_correlation[i, j] = cosine(counts[1][i], counts[1][j])
distance_correlation = distance_correlation + distance_correlation.T  # Make the matrix symmetric 
distance_correlation[np.isnan(distance_correlation)] = 1.0  # Assume that zero count individuals are maximally dissimilar
distance_correlation[np.isinf(distance_correlation)] = 1.0
df_similarity = pd.DataFrame(data=counts[1], index=experiment.Ants)
f_similarity = "spatial_fidelity_similarity_{}_{}_{}.csv".format(experiment.Name, t_start, t_end)
df_similarity.to_csv(f_similarity, index=experiment.Ants, columns=experiment.Ants)

Optionally: Create a similarity dendrogram and visualize it.

In [ ]:
fig, ax = plt.subplots()
Y = sch.linkage(distance_correlation, method='centroid')
Z = sch.dendrogram(Y, orientation='right')
index = Z['leaves']
distance_correlation = distance_correlation[index, :]
distance_correlation = distance_correlation[:, index]
fig, ax = plt.subplots()
ax.imshow(distance_correlation, cmap=plt.get_cmap('plasma'))